In [1]:
from pqp.variable import make_vars, Variable
from pqp.graph import Graph
from pqp.expression import Expression, Marginal, P, Product, Quotient

import matplotlib.pyplot as plt
import numpy as np 
import pandas as pd

In [2]:
x, y, z = make_vars("xyz")
g = Graph([
    z <= x,
    y <= z,
    x & y
])
estimand = g.idc([y], [x])
estimand.display()

<IPython.core.display.Math object>

In [3]:
I = object()
def p(*args):
    acc = vars = []
    given = []
    for arg in args:
        if arg is I:
            acc = given
        elif not isinstance(arg, Variable):
            raise ValueError("Expected Variable")
        else:
            acc.append(arg)
    return P(vars, given)

V = Variable

In [4]:
class attrdict(dict):
    def __getattr__(self, key):
        return self[key]

class CategoricalDistribution:
    def __init__(self, data, observed=None, prior=5):
        self.data = data
        self.n = data.shape[0]
        self.vars = attrdict({col: V(col) for col in data.columns})

        observed = observed or list(data.columns)
        domain = 1
        for obs in observed:
            domain *= len(np.unique(data[obs]))
        prior_observations = domain * prior

        self.chance_prior = prior_observations / (self.n + prior_observations)
        self.prior = self.chance_prior / domain
    
    def approx_p(self, expr: P, context):
        data = self.data
        for cond in expr.given:
            data = data[data[cond.name] == context[cond.name]]

        mask = np.ones(data.shape[0], dtype=bool)
        for var in expr.vars:
            if var.name not in context:
                raise ValueError(f"Missing context for {var.__repr__()}")
            mod = data[var.name] == context[var.name]
            mask = mask & mod
                
        return (mask.sum() / data.shape[0])*(1-self.chance_prior) + self.prior
    
    def approx_marginal(self, expr: Marginal, context):
        acc = 0
        if expr.sub:
            summation_var = expr.sub[0].name
            expr = expr.expr if len(expr.sub) == 1 else Marginal(expr.sub[1:], expr.expr)
            for val in np.unique(self.data[summation_var]):
                acc += self.approx(expr, {summation_var: val, **context})
        return acc
    
    def approx_product(self, expr: Product, context):
        acc = 1
        for e in expr.expr:
            acc *= self.approx(e, context)
        return acc
    
    def approx_quotient(self, expr: Quotient, context):
        return self.approx(expr.numer, context) / self.approx(expr.denom, context)

    def approx(self, expr, context = None):
        context = context or {}
        if isinstance(expr, P):
            res = self.approx_p(expr, context)
        elif isinstance(expr, Marginal):
            res = self.approx_marginal(expr, context)
        elif isinstance(expr, Product):
            res = self.approx_product(expr, context)
        elif isinstance(expr, Quotient):
            res = self.approx_quotient(expr, context)
        else:
            raise ValueError("Unsupported expression type: {}".format(type(expr)))
        return res

In [ ]:
dist = CategoricalDistribution(df)
est = p(dist.vars.outcome, I, dist.vars.treatment, dist.vars.)
dist.approx(estimand)

In [28]:
np.random.seed(137)
n = 10

severity = np.random.binomial(3, 0.5, size=n)
doctor_noise = np.random.binomial(1, 0.1, size=n)
print((severity > 1).astype(int))
print(doctor_noise)
treatment = ((severity > 1) != doctor_noise).astype(int)

# print(treatment)

pathway_predisposition = 0# np.random.binomial(2, 0.5, size=n)
pathway = pathway_predisposition - treatment

outcome_noise = 0#np.random.binomial(2, 0.5, size=n)
outcome = pathway + severity + outcome_noise

model_vars = {}
for name in ["severity", "treatment", "pathway", "outcome"]:
    model_vars[name] = Variable(name)

df = pd.DataFrame({
    "severity": severity,
    "treatment": treatment,
    "pathway": pathway,
    "outcome": outcome,
})

df.head(10)

[1 0 1 1 0 1 1 0 1 0]
[1 0 0 0 0 0 0 0 0 0]


,severity,treatment,pathway,outcome
0,3,0,0,3
1,0,0,0,0
2,2,1,-1,1
3,2,1,-1,1
4,1,0,0,1
5,2,1,-1,1
6,3,1,-1,2
7,0,0,0,0
8,2,1,-1,1
9,1,0,0,1


In [29]:
treat_outcome = df.outcome[df.treatment == 1].mean()
control_outcome = df.outcome[df.treatment == 0].mean()

print(f"treat outcome: {treat_outcome}")
print(f"control outcome: {control_outcome}")
print(f"treatment effect (naive): {treat_outcome - control_outcome}")

treat outcome: 1.2
control outcome: 1.0
treatment effect (naive): 0.19999999999999996


In [46]:
class Model:
    def __init__(self, dist, graph):
        self.dist = dist
        self.graph = graph
    
    def estimand(self, y, x=None, z=None):
        if x == None:
            x = []
        if z == None:
            z = []
        return self.graph.idc(y=[V(y)], x=[V(i) for i in x], z=[V(i) for i in z])
    
    def expectation(self, var, do=None, given=None):
        do = do or {}
        given = given or {}
        estimand = self.estimand(var, do.keys(), given.keys())

        acc = 0
        prob_acc = 0
        for val in np.unique(dist.data[var]):
            prob = self.dist.approx(estimand, {var: val, **do, **given})
            prob_acc += 1
            acc += prob * val
            print(prob, val)
        
        if abs(prob_acc - 1) > 1e-6:
            raise Exception(f"Probabilities do not sum to 1 (eps ~= {abs(prob_acc - 1)})")

        return acc

In [47]:
prior = 0.01

In [48]:
dist = CategoricalDistribution(df, observed=["treatment", "outcome", "pathway"], prior=prior)
graph = Graph([
    dist.vars.pathway <= dist.vars.treatment,
    dist.vars.outcome <= dist.vars.pathway,
    dist.vars.outcome & dist.vars.treatment,
])
model = Model(dist, graph)

print(dist.chance_prior)
model.estimand("outcome", ["treatment"]).display()

outcome_control = model.expectation("outcome", do={"treatment": 0})
outcome_treat = model.expectation("outcome", do={"treatment": 1})
print(f"treat outcome: {outcome_treat}")
print(f"control outcome: {outcome_control}")
print(f"treatment effect (my model): {outcome_treat - outcome_control}")

0.015748031496062992


<IPython.core.display.Math object>

0.39763779527559057 0
0.39763779527559057 1
0.003937007874015748 2
0.20078740157480315 3


Exception: Probabilities do not sum to 1 (eps ~= 3)

In [49]:
dist = CategoricalDistribution(df, observed=["treatment", "outcome", "pathway"], prior=prior)
graph = Graph([
    dist.vars.pathway <= dist.vars.treatment,
    dist.vars.outcome <= dist.vars.pathway,
])
model = Model(dist, graph)

print(dist.chance_prior)
model.estimand("outcome", ["treatment"]).display()

outcome_control = model.expectation("outcome", do={"treatment": 0})
outcome_treat = model.expectation("outcome", do={"treatment": 1})
print(f"treat outcome: {outcome_treat}")
print(f"control outcome: {outcome_control}")
print(f"treatment effect (bad model): {outcome_treat - outcome_control}")

0.015748031496062992


<IPython.core.display.Math object>

0.4031936127744511 0
0.4031936127744511 1
0.003992015968063872 2
0.20359281437125748 3


Exception: Probabilities do not sum to 1 (eps ~= 3)

In [50]:
dist = CategoricalDistribution(df, prior=prior/4)
graph = Graph([
    # dist.vars.pathway <= dist.vars.treatment,
    # dist.vars.outcome <= dist.vars.pathway,
    dist.vars.outcome <= dist.vars.treatment,
    dist.vars.treatment <= dist.vars.severity,
    dist.vars.outcome <= dist.vars.severity,
])
model = Model(dist, graph)

print(dist.chance_prior)
model.estimand("outcome", ["treatment"]).display()

outcome_control = model.expectation("outcome", do={"treatment": 0})
outcome_treat = model.expectation("outcome", do={"treatment": 1})
print(f"treat outcome: {outcome_treat}")
print(f"control outcome: {outcome_control}")
print(f"treatment effect (god): {outcome_treat - outcome_control}")

0.015748031496062992


<IPython.core.display.Math object>

0.5917808824395704 0
0.5917808824395704 1
0.394930488738783 2
0.5915354330708662 3


Exception: Probabilities do not sum to 1 (eps ~= 3)